In [1]:
import cv2 
from keras.models import load_model
import numpy as np

In [2]:
model = load_model('model-013.model')
face_clf = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [4]:
video = cv2.VideoCapture(0)
label_dict = {0: 'Without Mask', 1: 'With Mask'}
color_dict = {0: (0,0,255), 1:(0,255,0)}

while(True):
    ret, frame = video.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_clf.detectMultiScale(gray, 1.3, 5)
    
    for x, y, w, h in faces:
        
        face_img = gray[y:y+w,x:x+w]
        resized = cv2.resize(face_img, (100,100))
        normalized = resized/255.0
        reshaped = np.reshape(normalized, (1,100,100,1))
        reshaped = np.vstack([reshaped])
        result = model.predict(reshaped)
        
        label = np.argmax(result, axis=1)[0]
        
        cv2.rectangle(frame, (x,y), (x+w, y+h), color_dict[label], 2)
        cv2.rectangle(frame, (x, y-40), (x+w, y), color_dict[label], -1)
        cv2.putText(
        frame, label_dict[label],
            (x, y-10),
            cv2.FONT_HERSHEY_TRIPLEX, 0.8, (255,255,255), 2
        )    
    cv2.imshow('Live', frame)
    key = cv2.waitKey(1)
    
    if(key==27):
        break
               
cv2.destroyAllWindows()
video.release() 